<a href="https://colab.research.google.com/github/Bae-hong-seob/University-3-2/blob/main/Big_data_process_and_application/3_Dataframe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark
!apt install openjdk-8-jdk-headless -qq

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 58 kB/s 
     |████████████████████████████████| 199 kB 77.0 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=7e18a46ad6e22099a07c32039474569de4c7ba71eae50593b2832883d4d9c2d5
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upg

In [2]:
import os

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

In [4]:
import pyspark
from pyspark.sql import *

# 두 줄은 붙어서 작성되어야한다.
import pyspark.sql.functions as functions 
from pyspark.sql.types import *
from pyspark import SparkContext, SparkConf


In [5]:
sc = pyspark.SparkContext()
spark = SparkSession.builder.getOrCreate()

In [7]:
# strinJSONRDD RDD생성

stringJSONRDD = sc.parallelize(('''{'id':'123', 'name':'katie', 'age':19,'eyeColor':'brown'}''', 
                                '''{'id':'234', 'name':'Michael', 'age':22,'eyeColor':'green'}''',
                                '''{'id':'345', 'name':'Simone', 'age':23,'eyeColor':'blue'}'''))

In [8]:
# RDD를 Dataframe 객체로 변환
swimmersJSON = spark.read.json(stringJSONRDD)

In [9]:
swimmersJSON.show()

+---+--------+---+-------+
|age|eyeColor| id|   name|
+---+--------+---+-------+
| 19|   brown|123|  katie|
| 22|   green|234|Michael|
| 23|    blue|345| Simone|
+---+--------+---+-------+



ㄴ RDD 형태를 Dataframe 형태로 변환하여 출력함. show()함수 사용

In [16]:
# SQL 쿼리를 통해 데이터 확인.
swimmersJSON.createOrReplaceTempView('swimmersJSON')
print(spark.sql('select * from swimmersJSON where age>20').collect())

[Row(age=22, eyeColor='green', id='234', name='Michael'), Row(age=23, eyeColor='blue', id='345', name='Simone')]


ㄴ select는 레코드 추출. 따라서 조건문은 where절에 삽입

In [15]:
print('---------------------------------------')
spark.sql('select age>20 from swimmersJSON').collect()

---------------------------------------


[Row((age > 20)=False), Row((age > 20)=True), Row((age > 20)=True)]

In [17]:
swimmersJSON.printSchema()

root
 |-- age: long (nullable = true)
 |-- eyeColor: string (nullable = true)
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)



ㄴ 구조-- 칼럼명: 자료형 (Null가능성)

# 스키마 변경하기 (직접 만들기)

In [18]:
stringCSVRDD = sc.parallelize([(123,'Katie',19,'brown'),
                               (234,'Michael', 22, 'green'),
                               (345, 'Simone', 23, 'blue')])

In [19]:
# 스키마 명시(=지정)
# 세번째 parameter(=null 값 허용여부) True: Null값이 들어와도 ok
schema = StructType([
    StructField('id', LongType(), True),
    StructField('name', StringType(), True),
    StructField('age', LongType(), True),
    StructField('eyeColor', StringType(), True)
])

In [20]:
# 데이터만 존재하는 stringCSVRDD에 지정한 스키마(shcema) 적용
swimmers = spark.createDataFrame(stringCSVRDD, schema)

In [21]:
# shcema 적용된거 확인
swimmers.printSchema()

root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- age: long (nullable = true)
 |-- eyeColor: string (nullable = true)



# DataFrame 다뤄보기

In [22]:
# 행의 개수 출력
swimmers.createOrReplaceTempView('swimmers')
spark.sql('select count(1) from swimmers').show()

+--------+
|count(1)|
+--------+
|       3|
+--------+



In [23]:
# 필터문(=dataframe) 사용하기 sql where절이랑 똑같음
swimmers.select('id','age').filter('age=22').show()

+---+---+
| id|age|
+---+---+
|234| 22|
+---+---+



In [24]:
spark.sql('select id, age from swimmers where age=22').show()

+---+---+
| id|age|
+---+---+
|234| 22|
+---+---+



In [25]:
# filter(조건문) 사용2
swimmers.select('name','eyeColor').filter("eyeColor like 'b%'").show() #이때 " '' " 큰따음표, 작은따음표로 어디까지 문자열인지 구분해줘야함

+------+--------+
|  name|eyeColor|
+------+--------+
| Katie|   brown|
|Simone|    blue|
+------+--------+



In [26]:
spark.sql('select name, eyeColor from swimmers where eyeColor like "b%"').show()

+------+--------+
|  name|eyeColor|
+------+--------+
| Katie|   brown|
|Simone|    blue|
+------+--------+

